In [85]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import numpy as np
import matplotlib.pyplot as plt
from dataloaders import load_data, make_loaders, append_SOS

import sys
sys.path.insert(0, '/scratch/users/udemir15/ELEC491/bassline_transcription')
from utilities import *

from bassline_transcriber.transcription import NN_output_to_MIDI_file, replace_sustain
from tensorflow.keras.metrics import SparseCategoricalAccuracy

SEED = 27

np.random.seed(SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
with open('/scratch/users/udemir15/ELEC491/bassline_transcription/data/metadata/train_misc_names.txt', 'r') as infile:
    train_misc_names = infile.read().split('\n')
with open('/scratch/users/udemir15/ELEC491/bassline_transcription/data/metadata/val_misc_names.txt', 'r') as infile:
    val_misc_names = infile.read().split('\n')

In [73]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential

In [143]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class ResidualBlock(layers.Layer):
    def __init__(self, channel, downsample=True):
        super().__init__()
        strides = 2 if downsample else 1
        self.layers = Sequential([layers.Conv1D(channel, 4, activation=None, padding="same", use_bias=False),
                        layers.BatchNormalization(),
                        layers.Activation("relu"),
                        layers.Conv1D(channel, 4, activation=None, padding="same", strides=strides, use_bias=False),
                        layers.BatchNormalization()])
        
        self.shortcut = Sequential([
            layers.Conv1D(channel, 1, activation=None, padding="same", strides=strides, use_bias = False),
            layers.BatchNormalization()])
            
    def call(self, x):
        x = self.layers(x)
        h = self.shortcut(x)
        x = layers.add([h,x])
        x = layers.Activation("relu")(x)
        return x
    
class ResidualStack(layers.Layer):
    def __init__(self, channel, n_blocks=3, downsample=True):
        super().__init__()
        self.layers = Sequential([ResidualBlock(channel, downsample=False)]*(n_blocks-1) +[ResidualBlock(channel, downsample)])
        
    def call(self, data):
        return self.layers(data)
    
class Encoder(layers.Layer):
    
    def __init__(self, embedding_size, embedding_dim, latent_dim):
        
        super().__init__()
        
        self.embedding = layers.Embedding(embedding_size, embedding_dim)
    
        self.layer = Sequential([ResidualStack(32),
                                 ResidualStack(64),
                                 ResidualStack(128),
                                layers.Flatten(),
                                layers.Dense(16, activation="relu")])
               
        self.mean_layer = layers.Dense(latent_dim, name="z_mean")
        self.var_layer = layers.Dense(latent_dim, name="z_log_var")
        
        self.sampling = Sampling()
        
    def call(self, x):
        
        x = self.embedding(x)
        
        x = self.layer(x)
        
        z_mean = self.mean_layer(x)
        z_log_var = self.var_layer(x)
        
        z = self.sampling([z_mean, z_log_var])
        
        return z_mean, z_log_var, z 

class Decoder(layers.Layer):
    
    def __init__(self, input_, output_dim):
        
        super().__init__()
        self.layers = Sequential([layers.Dense(input_*8, activation="relu"),
                layers.Reshape((8, input_)),
                layers.Conv1DTranspose(128, 4, activation="relu", strides=2, padding="same"),
                ResidualStack(128, downsample=False),
                layers.Conv1DTranspose(64, 4, activation="relu", strides=1, padding="same"),
                ResidualStack(64, downsample=False),
                layers.Conv1DTranspose(32, 4, activation="relu", strides=1, padding="same"),
                ResidualStack(32, downsample=False),
                layers.Conv1DTranspose(output_dim, 4, activation="softmax", padding="same")])
        
    def call(self, x):        
        return self.layers(x)

In [144]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)        
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.accuracy_tracker = keras.metrics.Mean(name="acc")
        
        self.acc_fn = SparseCategoricalAccuracy()

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.accuracy_tracker,
        ]
    
    def call(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        return z_mean, z_log_var, z, reconstruction

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean(
               keras.losses.SparseCategoricalCrossentropy()(tf.cast(data, dtype=tf.float32), reconstruction)
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        acc=self.acc_fn(data, reconstruction)
        self.accuracy_tracker.update_state(acc)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "acc": self.accuracy_tracker.result()
        }

In [145]:
M = 8

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]',
               'dataset_name': 'TechHouse_bassline_representations_4020',
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params, False)

X = replace_sustain(X, 25)
X[X<9] += 12
X[X>=21] -= 12
X -= 9

X = X[[title not in val_misc_names+train_misc_names for title in titles]]
K = int(X.max()+1) # Number of classes, assumes consecutive [0,max] inclusive
X = X.reshape(-1, 16, 1)
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

Number of classes: 12
Sequence Length: 16
Number of data points: 14480


In [146]:
encoder = Encoder(K, 128, 4096)
decoder = Decoder(64, K)

model = VAE(encoder, decoder)
model.compile(optimizer=keras.optimizers.Adam())
model(X[:1])
model.summary()
model.fit(X, epochs=300, batch_size=16)

Model: "vae_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_35 (Encoder)         multiple                  537360    
_________________________________________________________________
decoder_33 (Decoder)         multiple                  2565612   
Total params: 3,102,982
Trainable params: 3,097,596
Non-trainable params: 5,386
_________________________________________________________________
Epoch 1/300
905/905 [==============================] - 19s 16ms/step - loss: 1.8366 - reconstruction_loss: 1.8061 - kl_loss: 4.8535e-05 - acc: 0.4777
Epoch 2/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7944 - reconstruction_loss: 1.7923 - kl_loss: 2.2914e-05 - acc: 0.4812
Epoch 3/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7895 - reconstruction_loss: 1.7908 - kl_loss: 2.1873e-05 - acc: 0.4814
Epoch 4/300
905/905 [==============================] - 14s

905/905 [==============================] - 14s 16ms/step - loss: 1.7845 - reconstruction_loss: 1.7878 - kl_loss: 1.5364e-05 - acc: 0.4819
Epoch 52/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7864 - reconstruction_loss: 1.7879 - kl_loss: 1.5056e-05 - acc: 0.4819
Epoch 53/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7932 - reconstruction_loss: 1.7878 - kl_loss: 1.4706e-05 - acc: 0.4819
Epoch 54/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7940 - reconstruction_loss: 1.7877 - kl_loss: 1.4628e-05 - acc: 0.4819
Epoch 55/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7855 - reconstruction_loss: 1.7878 - kl_loss: 1.4489e-05 - acc: 0.4819
Epoch 56/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7903 - reconstruction_loss: 1.7879 - kl_loss: 1.4410e-05 - acc: 0.4819
Epoch 57/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7839 - reconstruction_loss:

905/905 [==============================] - 14s 16ms/step - loss: 1.7847 - reconstruction_loss: 1.7877 - kl_loss: 5.2931e-06 - acc: 0.4819
Epoch 106/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7930 - reconstruction_loss: 1.7875 - kl_loss: 4.9737e-06 - acc: 0.4819
Epoch 107/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7837 - reconstruction_loss: 1.7876 - kl_loss: 4.9720e-06 - acc: 0.4819
Epoch 108/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7916 - reconstruction_loss: 1.7875 - kl_loss: 4.9619e-06 - acc: 0.4819
Epoch 109/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7892 - reconstruction_loss: 1.7876 - kl_loss: 4.8429e-06 - acc: 0.4819
Epoch 110/300
905/905 [==============================] - 14s 16ms/step - loss: 1.7899 - reconstruction_loss: 1.7876 - kl_loss: 4.8124e-06 - acc: 0.4819
Epoch 111/300
905/905 [==============================] - 17s 18ms/step - loss: 1.7870 - reconstruction

KeyboardInterrupt: 